In [ ]:
# library(tidyverse)

# aargang <- 2021
# bereg_filsti <- paste0("/ssb/stamme04/bereg/person/wk24/bosatte_koorfil_g", aargang, "m01d01eslep.sas7bdat")

# bereg <- haven::read_sas(bereg_filsti)

# nrow(bereg)

# bereg <- bereg %>%
# dplyr::select(bydel, gkrets, X_KOORDINAT, X_KOORDINAT)

# head(bereg)

# bereg_grunnkrets <- bereg %>%
# group_by(gkrets) %>%
# tally()

# nrow(bereg_grunnkrets)

### Laster inn pakker 

In [ ]:
sf::sf_use_s2(FALSE)

library(tidyverse)
library(readxl)
library(klassR)
library(sf)
library(leaflet)

### Velger årgang og kobler til X-disken

In [ ]:
aargang <- 2021

# Logg på for å få tilgang til data fra X-disken
system("kinit", input = getPass::getPass("Skriv inn Windows-passord"))

### Henter befolkningstall fra tabell 04317

OBS: erstatt med egen fil laget fra befolkningsregisteret? (for å få med under og over 18 år?)

In [ ]:
T04317 <- PxWebApiData::ApiData(04317, ContentsCode = "Personer", 
                                Grunnkretser = T, 
                                Tid = as.character(aargang)) [[2]] %>%
  dplyr::filter(!is.na(value)) %>%
  dplyr::rename(grunnkretsnummer = Grunnkretser, 
                personer = value) %>%
  dplyr::select(grunnkretsnummer, personer)

### Opptaksområder KLASS

In [ ]:
opptak <- readxl::read_excel("/ssb/bruker/rdn/Opptaksområder HF.xlsx")
opptak$kommunenummer <- gsub(".*if address = ", "", opptak$opptaksomrade)
opptak$kommunenummer <- substr(opptak$kommunenummer, 1, 4)
opptak$opptak <- sub('.*opptaksomr =', '', opptak$opptaksomrade)
opptak$opptak <- gsub('"', "", opptak$opptak)
opptak$opptak <- gsub(';', "", opptak$opptak)
opptak$opptak <- stringr::str_trim(opptak$opptak)
opptak["opptaksomrade"] <- NULL

KLASS_kommune <- klassR::GetKlass(131, output_level = 1, date = c(paste0(aargang, "-01-01"))) %>%
  dplyr::rename(kommunenummer = code,
                kommunenavn = name) %>%
  dplyr::filter(kommunenummer != "9999") %>%
  dplyr::left_join(opptak, by = "kommunenummer") %>%
  dplyr::mutate(HF = case_when(
    opptak == "Oslo" ~ "OSLO UNIVERSITETSSYKEHUS HF",
    opptak == "Stavanger" ~ "HELSE STAVANGER HF",
    opptak == "Fonna" ~ "HELSE FONNA HF",
    opptak == "Møre-Romsdal" ~ "HELSE MØRE OG ROMSDAL HF",
    opptak == "Nordland" ~ "NORDLANDSSYKEHUSET HF",
    opptak == "UNN" ~ "UNIVERSITETSSYKEHUSET NORD-NORGE HF",
    opptak == "Helgeland" ~ "HELGELANDSSYKEHUSET HF",
    opptak == "Østfold" ~ "SYKEHUSET ØSTFOLD HF",
    opptak == "V-Viken" ~ "VESTRE VIKEN HF",
    opptak == "Ahus" ~ "AKERSHUS UNIVERSITETSSYKEHUS HF",
    opptak == "Innlandet" ~ "SYKEHUSET INNLANDET HF",
    opptak == "Vestfold" ~ "SYKEHUSET I VESTFOLD HF",
    opptak == "Telemark" ~ "SYKEHUSET TELEMARK HF",
    opptak == "Sørlandet" ~ "SØRLANDET SYKEHUS HF",
    opptak == "Bergen" ~ "HELSE BERGEN HF",
    opptak == "Sogn-Fjordane" ~ "HELSE FØRDE HF", # OBS?
    opptak == "St.Olav" ~ "ST OLAVS HOSPITAL HF",
    opptak == "N-Trøndelag" ~ "HELSE NORD-TRØNDELAG HF",
    opptak == "Finnmark" ~ "FINNMARKSSYKEHUSET HF",
    TRUE ~ ""
  )) %>%
  dplyr::select(kommunenummer, kommunenavn, HF) %>%
  dplyr::mutate(HF = case_when(
    kommunenummer == "4623" ~ "HARALDSPLASS DIAKONALE SYKEHUS AS", 
    TRUE ~ HF
  ))

KLASS_grunnkrets <- klassR::GetKlass(131, correspond = 1) %>%
  dplyr::rename(kommunenummer = sourceCode, 
                grunnkretsnummer = targetCode) %>%
  dplyr::filter(grunnkretsnummer != "03019999") %>%
  dplyr::select(kommunenummer, grunnkretsnummer) %>%
  dplyr::left_join(KLASS_kommune, by = "kommunenummer") %>%
  dplyr::filter(kommunenummer != "0301") %>%
  dplyr::select(grunnkretsnummer, HF)

### Bydel fix ###
KLASS_bydel_oslo <- klassR::GetKlass(103, correspond = 1) %>%
  dplyr::rename(bydelsnummer = sourceCode, 
                grunnkretsnummer = targetCode) %>%
  dplyr::select(bydelsnummer, grunnkretsnummer) %>%
  dplyr::filter(substr(grunnkretsnummer, 1, 4) == "0301") %>%
  dplyr::mutate(HF = case_when(
    substr(bydelsnummer, 5, 6) %in% c("10", "11", "12") ~ "AKERSHUS UNIVERSITETSSYKEHUS HF",
    substr(bydelsnummer, 5, 6) %in% c("01", "02", "03", "04") ~ "LOVISENBERG DIAKONALE SYKEHUS AS",
    substr(bydelsnummer, 5, 6) %in% c("05", "06", "07") ~ "DIAKONHJEMMET SYKEHUS AS",
    TRUE ~ "OSLO UNIVERSITETSSYKEHUS HF"
  )) %>%
  dplyr::select(grunnkretsnummer, HF)

opptaksomrader_HF <- rbind(KLASS_grunnkrets, KLASS_bydel_oslo) %>%
  dplyr::left_join(T04317, by = "grunnkretsnummer")
nrow(opptaksomrader_HF)

opptaksomrader_HF$personer[is.na(opptaksomrader_HF$personer)] <- 0

### Laster inn kart (grunnkrets)

In [ ]:
# sf::st_layers(paste0("Q:/GIS-ressurssenter/KartdataQ/BasisKart/kart", aargang, ".gdb"))

# grunnkrets_kart <- sf::read_sf(paste0("/ssb/x_disk/Felles/GIS-ressurssenter/KartdataQ/BasisKart/kart", aargang, ".gdb"), layer = paste0("ABAS_grunnkrets_utenhav_", aargang)) %>%
grunnkrets_kart <- sf::read_sf(paste0("/ssb/x_disk/Felles/GIS-ressurssenter/KartdataQ/BasisKart/kart", aargang, ".gdb"), layer = paste0("ABAS_grunnkrets_flate_", aargang)) %>%
  sf::st_transform(crs = 4326) %>%
  dplyr::rename(grunnkretsnummer = GRUNNKRETS) %>%
  dplyr::left_join(opptaksomrader_HF, by = "grunnkretsnummer")

ssb_farger <- klassR::GetKlass(614, output_style = "wide") %>%
  dplyr::rename(farge_nummer = code3, 
                HEX = name3, 
                farge = name2, 
                type = name1) %>%
  dplyr::select(-code1, -code2) %>%
  dplyr::filter(farge != "Hvit")

pal_HF <- leaflet::colorFactor(ssb_farger$HEX, domain = as.factor(grunnkrets_kart$HF))

### Lager kart over opptaksområder

In [ ]:
# Slår sammen geometrier per opptaksområde (HF)
opptaksomrader_HF <- grunnkrets_kart %>% 
  # filter(!HF %in% c("SYKEHUSET ØSTFOLD HF")) %>%
  dplyr::group_by(HF) %>%
  dplyr::summarise(SHAPE = sf::st_union(sf::st_make_valid(SHAPE)),
                   personer = sum(personer)) %>%
  dplyr::ungroup()

opptaksomrader_HF_leaflet <- leaflet::leaflet(options = leaflet::leafletOptions(zoomControl = FALSE)) %>% 
  leaflet::addTiles() %>%
  leaflet::addPolygons(stroke = F, data = opptaksomrader_HF,
                       # color = "green",
                       weight = 1,
                       fillColor = pal_HF(opptaksomrader_HF$HF),
                       fillOpacity = 0.5, smoothFactor = 0.5, 
                       popup = paste0("Opptaksområde: ", opptaksomrader_HF$HF, " / Befolkning: ", opptaksomrader_HF$personer)) %>%
  leaflet::addLegend("bottomright", pal = pal_HF, values = as.factor(opptaksomrader_HF$HF), opacity = 1)

opptaksomrader_HF_leaflet

htmlwidgets::saveWidget(opptaksomrader_HF_leaflet, file="./images/opptaksområder_HF.html", selfcontained=T)